In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from typing import *
import copy
from deeppavlov.dataset_readers.ontonotes_reader import OntonotesReader

2018-09-04 13:09:12.711 DEBUG in 'matplotlib.backends'['__init__'] at line 90: backend module://ipykernel.pylab.backend_inline version unknown


In [2]:
from src.fewshot_ner_viz_component.data_processing import *
from src.fewshot_ner_viz_component.utils import *
from src.fewshot_ner_viz_component.fewshot_ner_binary_classifier import *

2018-09-04 13:09:12.801 INFO in 'summarizer.preprocessing.cleaner'['textcleaner'] at line 37: 'pattern' package not found; tag filters are not available for English


In [3]:
dataset_orig = read_data()
ne_type = 'PERSON'
dataset = filter_dataset_by_ne_types(dataset_orig, ne_type)

Num of train sentences: 75187
Num of valid sentences: 9603
Num of test sentences: 9479
[(['Actions', 'had', 'to', 'be', 'taken', 'to', 'break', 'through', 'the', 'blockade', '.'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']), (['On', 'a', 'night', 'in', 'late', 'July', '1940', ',', 'the', 'atmosphere', 'in', 'Zhuanbi', 'Village', 'in', 'Shaanxi', 'was', 'unusual', '.'], ['O', 'B-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'I-DATE', 'O', 'O', 'O', 'O', 'B-GPE', 'I-GPE', 'O', 'B-GPE', 'O', 'O', 'O']), (['Villager', 'Xiao', 'Jianghe', 'has', 'a', 'vivid', 'memory', 'of', 'this', 'piece', 'of', 'history', '.'], ['O', 'B-PERSON', 'I-PERSON', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']), (['On', 'that', 'dark', 'night', ',', 'everyone', 'was', 'sleeping', 'when', 'human', 'voices', 'and', 'neighing', 'horses', 'were', 'heard', 'within', 'the', 'village', '.'], ['O', 'B-TIME', 'I-TIME', 'I-TIME', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '

In [4]:
N_EXAMPLES = 10
N_TEST_SENTENCES = 100
N_SAMPLES = 10
MODELS = ['ne_centroid', 'weighted_kNN', 'centroid_kNN', 'svm']
# MODELS = ['svm']
MODEL_PARAMS = {'ne_centroid': {'sim_type': 'cosine'},
               'ne_nearest': {'sim_type': 'cosine'},
               'weighted_kNN': {'k': 1, 'metric': 'cosine', 'use_class_weights': False, 'use_sim_weights': True},
               'centroid_kNN': {'k': 10, 'metric': 'dot_prod', 'use_class_weights': False},
               'svm': {}}

### Make samples

In [5]:
data_samples = []
for i in range(N_SAMPLES):
    tokens_train,tags_train = get_data_sample(dataset['train'], N_EXAMPLES)
    tokens_test,tags_test = get_data_sample(dataset['test'], min(N_TEST_SENTENCES, len(dataset['test'])))
    y_test = np.array([1 if t == 'T' else 0 for seq in tags_test for t in seq])
    data_samples.append({'train': {'tokens': tokens_train, 'tags': tags_train}, 'test': {'tokens': tokens_test, 'tags': tags_test, 'y': y_test}})

### Prepare embedder params

In [6]:
embedder_params=[{'elmo_params': {'trainable_cells': False}, 'name': 'Default ELMo', 'label': 'default'},
             {'elmo_params': {'trainable_cells': False, 'restore_path': "./model_params/elmo.ckpt"}, 'name': 'Learned ELMo layer aggregation coefficients', 'label': 'elmo_coef'},
             {'elmo_params': {'trainable_cells': True, 'restore_path': "./model_params/elmo.ckpt"}, 'name': 'ELMo coefficients + cell params', 'label': 'elmo_coef_and_cells'}]

In [7]:
f1_scores = {}
sims = {}
for i, embedder_params_cur in enumerate(embedder_params):
    print('============ {} ============'.format(embedder_params_cur['name']))
    elmo_params = embedder_params_cur['elmo_params']
    label = embedder_params_cur['label']
    tf.reset_default_graph()
    embedder = CompositeEmbedder(use_elmo=True, elmo_scale=1, cap_scale=1, use_cap_feat=False, use_glove=False, elmo_params=elmo_params)
    f1_scores[label] = {}
    for model in MODELS:
        f1_scores[label][model] = {'vals': [], 'avg': 0}
    for j, data_sample in enumerate(data_samples):
        print('Sample {}/{}'.format(j+1, N_SAMPLES))
        tokens_train = data_sample['train']['tokens']
        tags_train = data_sample['train']['tags']
        tokens_test = data_sample['test']['tokens']
        tags_test = data_sample['test']['tags']
        y_test = data_sample['test']['y']
        ner_bin_clf = FewshotNerBinaryClassifier(embedder)
        ner_bin_clf.train_on_batch(tokens_train, tags_train)
        results = ner_bin_clf.predict(tokens_test, MODELS, MODEL_PARAMS)
        for model in MODELS:    
            pred = results[model]['pred']
            f1 = f1_score(y_test, pred)
            f1_scores[label][model]['vals'].append(f1)
            print('Model: {}, F1 (tokens) = {:.2f}'.format(model, f1*100))
    print('Average F1 scores:')
    for model in MODELS:
        f1_avg = np.mean(f1_scores[label][model]['vals'])
        f1_scores[label][model]['avg'] = f1_avg
        print('Model: {}, avg F1 (tokens) = {:.2f}'.format(model, f1_avg*100))

============ Default ELMo ============
INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.


2018-09-04 13:09:15.811 INFO in 'tensorflow'['tf_logging'] at line 159: Using /tmp/tfhub_modules to cache modules.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


2018-09-04 13:09:16.988 INFO in 'tensorflow'['tf_logging'] at line 115: Saver not created because there are no variables in the graph to restore


Sample 1/10
Train
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

NE centroid similarity model
SVM classifier model
Weighted kNN model
k = 1, metric: cosine
[====================] 100%
NE centroid + words kNN similarity model
k = 10, metric: dot_prod
[====================] 100%
Model: ne_centroid, F1 (tokens) = 90.12
Model: weighted_kNN, F1 (tokens) = 89.51
Model: centroid_kNN, F1 (tokens) = 86.17
Model: svm, F1 (tokens) = 89.52
Sample 10/10
Train
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


2018-09-04 13:29:28.100 INFO in 'tensorflow'['tf_logging'] at line 115: Saver not created because there are no variables in the graph to restore


# ne: 23, # tokens: 277
n_samples: 277
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


2018-09-04 13:29:29.741 INFO in 'tensorflow'['tf_logging'] at line 115: Saver not created because there are no variables in the graph to restore


NE centroid similarity model
SVM classifier model
Weighted kNN model
k = 1, metric: cosine
[====================] 100%
NE centroid + words kNN similarity model
k = 10, metric: dot_prod
[====================] 100%
Model: ne_centroid, F1 (tokens) = 90.75
Model: weighted_kNN, F1 (tokens) = 89.07
Model: centroid_kNN, F1 (tokens) = 90.25
Model: svm, F1 (tokens) = 92.53
Average F1 scores:
Model: ne_centroid, avg F1 (tokens) = 87.79
Model: weighted_kNN, avg F1 (tokens) = 87.09
Model: centroid_kNN, avg F1 (tokens) = 86.42
Model: svm, avg F1 (tokens) = 87.40


In [8]:
print(f1_scores)

{'default': {'ne_centroid': {'vals': [0.736, 0.7822410147991543, 0.6774193548387096, 0.7747747747747749, 0.8042895442359248, 0.8138957816377171, 0.778494623655914, 0.780701754385965, 0.7826086956521738, 0.6756756756756755], 'avg': 0.7606101219656008}, 'weighted_kNN': {'vals': [0.6643835616438357, 0.8416666666666667, 0.7833827893175075, 0.836283185840708, 0.8585131894484412, 0.8354430379746836, 0.8089887640449438, 0.8088888888888888, 0.8551401869158879, 0.8353909465020577], 'avg': 0.812808121724362}, 'centroid_kNN': {'vals': [0.8523002421307506, 0.8660714285714286, 0.7753846153846154, 0.8995215311004785, 0.8591885441527446, 0.8984771573604061, 0.8539325842696629, 0.8542713567839195, 0.8803827751196172, 0.8752941176470588], 'avg': 0.8614824352520681}, 'svm': {'vals': [0.8340425531914893, 0.8522483940042828, 0.8294117647058824, 0.8699551569506727, 0.8932038834951457, 0.9026128266033254, 0.8453159041394337, 0.8755760368663594, 0.8943488943488944, 0.9391304347826086], 'avg': 0.8735845849088

In [9]:
labels = [par['label'] for par in embedder_params]
print('{:<8s}\t{:<8s}\t{:<8s}\t{:<8s}'.format('MODEL', embedder_params[0]['label'], embedder_params[1]['label'], embedder_params[2]['label']))
for model in MODELS:
    print('{:<8s}\t{:<8.2f}\t{:<8.2f}\t{:<8.2f}'.format(model, f1_scores[labels[0]][model]['avg']*100, f1_scores[labels[1]][model]['avg']*100, f1_scores[labels[2]][model]['avg']*100))

MODEL   	default 	elmo_coef	elmo_coef_and_cells
ne_centroid	76.06   	78.58   	87.79   
weighted_kNN	81.28   	81.61   	87.09   
centroid_kNN	86.15   	86.36   	86.42   
svm     	87.36   	86.58   	87.40   
